In [1]:
# import supabase
import os 
from datasets import Dataset 
from ragas.metrics import faithfulness, context_precision, answer_relevancy
from ragas import evaluate
import json
from openai import OpenAI

/Users/drshikaasher/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initiate Supabase Client
# supabase_client = supabase.create_client(supabase_url="redacted", supabase_key="redacted")

In [3]:
# import json

# page_size = 100
# last_id = 0
# all_data = []

# while True:
#     try:
#         # Fetch data using cursor-based pagination
#         response = supabase_client.table("llm-convo-monitor") \
#             .select("id", "convo") \
#             .eq("course_name", "cropwizard-1.5") \
#             .order("id") \
#             .gt("id", last_id) \
#             .limit(page_size) \
#             .execute()
        
#         # Check if the response is valid
#         if not response.data:
#             print("No data returned. Breaking the loop.")
#             break
        
#         # Append the fetched data to the result
#         all_data.extend(response.data)
        
#         # Update the last_id for the next iteration
#         last_id = response.data[-1]['id']
        
#         print(f"Fetched {len(response.data)} records. Last ID: {last_id}")
        
#     except json.JSONDecodeError as e:
#         print(f"JSON Decode Error: {str(e)}")
#         print("Response content:", response.content)
#         break
#     except Exception as e:
#         print(f"An error occurred: {str(e)}")
#         break

# # Print the total number of records fetched
# print(f"Total records fetched: {len(all_data)}")

# # Use all_data for further processing

In [4]:
# Question -> conversation.messages[0].content[0].text
# Answer -> conversation.messages[1].content
# contexts -> conversation.messages[0].contexts # this is an array of dicts

# metrics=[context_precision, faithfulness, answer_relevancy]

# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(all_data, f, ensure_ascii=False, indent=4)
# Load data from data.json

with open('data.json', 'r', encoding='utf-8') as f:
    all_data = json.load(f)  # Load only the first 10 objects

print(f"Loaded {len(all_data)} records from data.json")


Loaded 967 records from data.json


In [5]:
import traceback

os.environ["OPENAI_API_KEY"] = "redacted"

data_samples = {
    'question': [],
    'answer': [],
    'contexts': [],
    'reference': []
}

for index, item in enumerate(all_data):
    try:
        if len(item['convo']['messages']) < 2:
            continue
        
        question_content = item['convo']['messages'][0]['content']
        if not isinstance(question_content, list) or len(question_content) == 0 or 'text' not in question_content[0]:
            continue
        question = question_content[0]['text']
        if not isinstance(question, str):
            continue
        
        answer_content = item['convo']['messages'][1]['content']
        if isinstance(answer_content, list):
            answer = ' '.join([content.get('text', '') for content in answer_content if content.get('type') == 'text'])
        elif isinstance(answer_content, str):
            answer = answer_content
        else:
            continue
        contexts = item['convo']['messages'][0].get('contexts', [])
        
        data_samples['question'].append(question)
        data_samples['answer'].append(answer)
        flattened_contexts = [context['text'] for context in contexts]
        # print(flattened_contexts)
        data_samples['contexts'].append(flattened_contexts)
        data_samples['reference'].append('')  # Placeholder for reference
    except Exception as e:
        continue

# print(f"Processed items: question: {len(data_samples['question'])}, answer: {len(data_samples['answer'])}, contexts: {len(data_samples['contexts'])}, reference: {len(data_samples['reference'])}")

# Ensure all lists have the same length
min_length = min(len(data_samples[key]) for key in data_samples)
for key in data_samples:
    data_samples[key] = data_samples[key][:min_length]

# print(f"Final lengths: question: {len(data_samples['question'])}, answer: {len(data_samples['answer'])}, contexts: {len(data_samples['contexts'])}, reference: {len(data_samples['reference'])}")

try:
    dataset = Dataset.from_dict(data_samples)
    print("made dataset")
    score_r = evaluate(dataset,metrics=[faithfulness, answer_relevancy, context_precision])
    print("calculated score")
    print("final scores", score_r)
except Exception as e:
    print(f"An error occurred while creating the dataset: {str(e)}")
    traceback.print_exc()
    # for key, value in data_samples.items():
    #     print(f"{key}: {type(value)}, length: {len(value)}")
    #     if value:
    #         print(f"First {key} item type: {type(value[0])}")
    #         print(f"First {key} item: {value[0][:100]}...")  # Print first 100 characters

made dataset


Evaluating: 100%|██████████| 1503/1503 [1:26:44<00:00,  3.46s/it]


calculated score
final scores {'faithfulness': 0.7710, 'answer_relevancy': 0.7267, 'context_precision': 0.3007}
